# Quick plotting for SFR79 from Observations and Simulations

### Imports

In [ ]:
import numpy as np

from pathlib import Path
from datetime import datetime
import warnings

import matplotlib as mpl
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm, trange

import SiGMo as sgm

### Defining Star-forming Galaxy Main Sequence

Follows the definiton of Saintonge+2016, Eq. 5. They use SDSS DR7 with 0.01 < z < 0.05.

In [ ]:
def GMS_Saintonge2016(logMstar):
    '''computes the log10SFR for any given log10Mstar on Galaxy Main Seqence according to Saintonge+2016, Eq.5'''
    return ((-2.332) * logMstar + 0.4156 * logMstar**2 - 0.01828 * logMstar**3)

### Read-in

In [ ]:
project_dir = Path.cwd().parent
sfr79_dir = project_dir / 'data' / "SFR79_grids"
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.01-12.44.47"   # sim with mgas == SFR (also initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.04-01.48.44"   # sim with mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "backward"   # vintage mode simulation with full grid and mgas == SFR (also initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.08-08.50.16"   # sim with MLF=0.05, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.08-10.25.14"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.08-16.18.04"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.08-19.07.38"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.14-14.54.07"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.15-12.00.38"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.16-19.44.57"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.17-12.22.07" / "0_forward_800Myr"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.17-12.22.07" / "1_backward_800Myr"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.17-20.51.25" / "0_forward_800Myr"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.02.17-20.51.25" / "1_backward_800Myr"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.03.02-14.55.48" / "0_forward_800Myr"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.03.02-14.55.48" / "1_backward_800Myr"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.03.08-17.52.38" / "0_forward_800Myr_1e-4"   # sim with MLF=0.2, and mgas from sSFR (initially)
# snp_dir = project_dir / 'outputs' / '_tmp' / "2022.03.08-17.52.38" / "1_backward_800Myr_1e-4"   # sim with MLF=0.2, and mgas from sSFR (initially)
snp_dir = project_dir / 'outputs' / '_tmp' / "2022.03.10-21.01.29" / "0_backward_2Gyr_1e-3"   # sim with MLF=0.2, and mgas from sSFR (initially)
# plot_dir = project_dir / 'plots' / '_tmp' / snp_dir.name   # plot dir has now same datetime name as output dir
plot_dir = project_dir / 'plots' / '_tmp' / snp_dir.parent.name / snp_dir.name   # plot dir has now same datetime name as output dir
plot_dir.mkdir(exist_ok=True, parents=True)   # create plot dir only if necessary


# reading observational results
sfr79_medians = np.loadtxt(str(sfr79_dir / "SFR79_2dhist_medians.txt"))
mstar_mesh =  np.loadtxt(str(sfr79_dir / "SFR79_2dhist_binedges_mstar_mesh.txt"))
sfr_mesh = np.loadtxt(str(sfr79_dir / "SFR79_2dhist_binedges_sfr_mesh.txt"))
n_binned = np.loadtxt(str(sfr79_dir / "SFR79_2dhist_binnumbers.txt"))

# removing low-number bins (if wanted, as before when setting up the simulation)
# if not wanted: comment the next two lines out!
n_binned_min = 40
sfr79_medians = np.where(n_binned >= n_binned_min, sfr79_medians, np.nan)

#### Version for huge numbers of single snapshots per timestep
➡️ single_snapshot(s) = True when making/writing the snapshots

#### Version for one large multi snapshot per timestep
➡️ single_snapshot(s) = False when making/writing the snapshots

#### Version for *both*: huge numbers of single snapshots *or* one large multi snapshot per timestep
single_snapshot(s) = True when making/writing the snaps ➡️ set single_snapshots = True
single_snapshot(s) = False when making/writing the snaps ➡️ set single_snapshots = False

In [ ]:
# setting the snapshot type flag! False -> from one big Env snap; True -> from many indiv. snaps
single_snapshots = False

# reading simulation results

# get all file names
file_iter = snp_dir.iterdir()
file_list = list(file_iter)
n_files = len(file_list)

# how many timesteps?
if single_snapshots:
    # n_steps = 800
    n_steps = 799
else:
    n_steps = n_files

# how many objects per timestep?
n_envs = 1
n_halos = len(sfr79_medians.flat) - len(sfr79_medians[np.where(np.isnan(sfr79_medians))].flat)  # assuming same mstar-sfr pairs in the sims as in the obs
n_gals = n_halos

# make arrays here already
env_grid = np.empty(shape=(n_envs, n_steps), dtype=object)
halo_grid = np.empty(shape=(n_halos, n_steps), dtype=object)
gal_grid = np.empty(shape=(n_gals, n_steps), dtype=object)

# string elements to split and recognise file names
fn_start = "snp"
fn_middle_gal = "Galaxy"
fn_middle_halo = "Halo"
fn_middle_env = "Environment"
fn_end = ""
fn_separator = "_"

# # get all file names  (I do this earlier now, with len(list(file_iter))
# file_iter = snp_dir.iterdir()
# n_files = n_envs * n_steps if not single_snapshots else (n_envs + n_halos + n_gals) * n_steps  #hardcoded to Environment

# loop trough the files in the arbitrary order that they got in the iterator file_iter
# for file in tqdm(file_iter, total=(n_envs + n_halos + n_gals) * n_steps):
for file in tqdm(file_list, total=n_files):
    fn = file.name

    # check that fn is regular snapshot of time evolution, not other file (e.g. "final" snapshot etc)
    if fn.startswith(fn_start):
        # split the file name (not the whole path)
        fn_split = fn.split(fn_separator)

        # assign the different elements from file name to different counters (time and object/param combination)
        t = int(fn_split[0][len(fn_start):])
        i = int(fn_split[2]) if len(fn_split) == 4 else 0
        snp_type = fn_split[1]

        # make full file path with directories
        full_file_path = file.resolve()

        # differentiate between Galaxy, Halo and Environment files
        if snp_type == fn_middle_env:
            env_grid[...,t].flat[i] = sgm.Snapshot.load_from_disk(full_file_path)
        elif snp_type == fn_middle_halo:
            if single_snapshots:
                halo_grid[...,t].flat[i] = sgm.Snapshot.load_from_disk(full_file_path)
            else:
                warnings.warn(f"The type '{snp_type}' of snapshot '{file}' is not yet supported for multi_snapshot "
                              f"read in!\nSnapshot was not read in.")
        elif snp_type == fn_middle_gal:
            if single_snapshots:
                gal_grid[...,t].flat[i] = sgm.Snapshot.load_from_disk(full_file_path)
            else:
                warnings.warn(f"The type '{snp_type}' of snapshot '{file}' is not yet supported for multi_snapshot "
                              f"read in!\nSnapshot was not read in.")
        else:
            warnings.warn(f"The type '{snp_type}' of snapshot '{file}' was not recognised! Snapshot was not read in.")

In [ ]:
# re-create/populate halo_grid and galaxy_grid (for FIRST Environment only) if necessary
if not single_snapshots:
    for t, env_snp in enumerate(tqdm(env_grid[0])):  # HARDCODED to 1st Environment only!
        for i_halo, halo in enumerate(env_snp.data['halos']):
            halo_grid[i_halo, t] = sgm.Snapshot(halo)
            for i_gal, gal in enumerate(halo['galaxies']):
                gal_grid[i_halo, t] = sgm.Snapshot(gal)  # HARDCODED to 1 Galaxy per Halo!!

### SFR79

Compute the log SFR79 from the "actual" simulation SFR averages:

$$\mathrm{SFR79} = \frac{\mathrm{SFR}_\mathrm{5 Myr}}{\mathrm{SFR}_\mathrm{800 Myr}}$$

In [ ]:
SFR79_grid = np.empty(shape=(*gal_grid.shape[:-1], 3), dtype=object)

SFR79_grid_fl = SFR79_grid.reshape(-1, *SFR79_grid.shape[-1:])
gal_grid_fl = gal_grid.reshape(-1, *gal_grid.shape[-1:])

for i, gal_seq in enumerate(tqdm(gal_grid_fl)):
    # SFR79_grid_fl[i, 0] = np.sum(gal_grid_fl[i,:][gal_grid_fl[i,:].data['lookbacktime'] <= 0.005]['SFR'])
    # print(gal_grid_fl[i,:][gal_grid_fl[i,:].data['lookbacktime'] <= 0.005]['SFR'])
    lookbacktime_a = np.array([gal.data["lookbacktime"] for gal in gal_seq])
    SFR_a = np.array([gal.data["SFR"] for gal in gal_seq])

    # avrg over 5 Myr
    SFR7_a = SFR_a[lookbacktime_a <= 0.005]
    SFR7_avg = np.sum(SFR7_a) / len(SFR7_a)
    SFR79_grid_fl[i, 0] = SFR7_avg
    # print(SFR79_grid_fl[i, 0])

    # avrg over 800 Myr
    SFR9_a = SFR_a[lookbacktime_a <= 0.8]
    SFR9_avg = np.sum(SFR9_a) / len(SFR9_a)
    SFR79_grid_fl[i, 1] = SFR9_avg
    # print(SFR79_grid_fl[i, 1])

    # log10(SFR_5Myr / SFR_800Myr)
    SFR79_grid_fl[i, 2] = np.log10(SFR7_avg / SFR9_avg)
    # print(SFR79_grid_fl[i, 2])

print(SFR79_grid)

**select galaxies/halos: plot quantities and differences for individual param combinations over time**

In [ ]:
# define what to plot
x_type = 'lookbacktime'
y_type = 'SFR'
# which_objects = [0, 100, 200, 300, 400, 500, 600, 700, 800]
which_objects = [0, 50, 100, 150, 200, 250, 300]

# grab plotting data
x_data = []
y_data = []
label_l = []
for i in tqdm(which_objects):
    x_data.append([gal.data[x_type] for gal in gal_grid[i][:]])
    y_data.append([gal.data[y_type] for gal in gal_grid[i][:]])
    # y_data.append([halo.data[y_type] for halo in halo_grid[i][:]])
    label_l.append(r"$\mathrm{M}_\star$"+ f"= {gal_grid[i, 0].data['mstar']:.2e} \t SFR = {gal_grid[i, 0].data['SFR']/10**9:.2e}")

# initialise plot
fig, ax = plt.subplots(nrows=2, figsize=(9, 9), constrained_layout=True)

# plot actual values
for i, (x, y, label) in enumerate(zip(x_data, y_data, label_l)):
    ax[0].plot(x, y, label=label)


# plot one quantity like SFR79
for i, label in zip(which_objects, label_l):
    ax[1].plot(i, SFR79_grid[i, -1], 'o', label=label)
# # plot residual of directions[0] and directions[1]
# ax[1].plot(x,
#            [(2*(f - b)/(f + b)) if f+b != 0 else 0 for f, b in zip(y_data[0], y_data[1])])
# ax[1].axhline(0, ls=':')

# additional fig and axes config
# fig.suptitle(f'{y_type}: forward vs backward', fontsize=16)
fig.suptitle(f'{y_type}: comparing {len(which_objects)} simulated galaxies', fontsize=16)

# for a in ax:
#     a.invert_xaxis()
#     a.set_xlabel(x_type)
ax[0].invert_xaxis()
ax[0].set_xlabel(x_type)
ax[0].set_yscale('log')
ax[0].set_ylabel(y_type)

ax[1].set_xlabel('simulation number (arb. index)')
# ax[1].set_ylabel(f'Δ{y_type}(t) / mean({y_type}' + r'$_\mathrm{fw}$, ' + f'{y_type}' + r'$_\mathrm{bw}$)')
ax[1].set_ylabel(f'log SFR79')

ax[1].legend()


# fig.savefig(project_dir / f'/plots/_tmp/{y_type}_forward_vs_backward_comparison.png')
fig.savefig(plot_dir / f'comparing_{y_type}_of_{len(which_objects)}_simulated_galaxies'
                       f'_{datetime.now().strftime("%Y.%m.%d-%H.%M.%S")}.png')

**all galaxies/halos: plot quantities and differences for individual param combinations over time**

In [ ]:
# define what to plot
x_type = 'lookbacktime'
y_type = 'SFR'
# c_type = 'mtot'
# c_type = 'mtot_over_mstar'
c_type = 'mtot_over_mgas'
c_snp = 0
# c_snp = -1
c_alpha = 0.25
# which_objects = [0, 100, 200, 300, 400, 500, 600, 700, 800]
which_objects = range(0, len(gal_grid), 1)

# grab plotting data
x_data = []
y_data = []
c_data = []
label_l = []
for i in tqdm(which_objects):
    x_data.append([gal.data[x_type] for gal in gal_grid[i][:]])
    y_data.append([gal.data[y_type] for gal in gal_grid[i][:]])
    # y_data.append([halo.data[y_type] for halo in halo_grid[i][:]])
    if gal_grid[i][c_snp].data['mgas'] == 0 and halo_grid[i][c_snp].data['mtot'] != 0:
        _c_data_i = [np.nan]
    else:
        _c_data_i = [halo_grid[i][c_snp].data['mtot'] / gal_grid[i][c_snp].data['mgas']]
    # c_data.append([halo_grid[i][c_snp].data[c_type]])
    # c_data.append([halo_grid[i][c_snp].data['mtot'] / gal_grid[i][c_snp].data['mstar']])
    c_data.append(_c_data_i)
    label_l.append(r"$\mathrm{M}_\star$"+ f"= {gal_grid[i, 0].data['mstar']:.2e} \t SFR = {gal_grid[i, 0].data['SFR']/10**9:.2e}")


# normalise the colour data to make colour the data according to it (e.g. total halo mass mtot)
c_data_range = (np.nanmin(c_data), np.nanmax(c_data))
cmap = mpl.cm.viridis_r
# cmap = mpl.cm.tab20b
norm = mpl.colors.LogNorm(vmin=c_data_range[0], vmax=c_data_range[1])
mapper = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)


# initialise plot
fig, ax = plt.subplots(nrows=2, figsize=(9, 9), constrained_layout=True)

# plot actual values
for i, (x, y, c, label) in enumerate(zip(x_data, y_data, c_data, label_l)):
    c_mapped = mapper.to_rgba(c)
    c_mapped[:, -1] = c_alpha
    ax[0].plot(x, y, c=c_mapped, label=label)


# plot one quantity like SFR79
for i, c, label in zip(which_objects, c_data, label_l):
    c_mapped = mapper.to_rgba(c)
    c_mapped[:, -1] = c_alpha
    # ax[1].plot(i, SFR79_grid[i, -1], c=c_mapped, marker='o', label=label)
    ax[1].plot(c_data[i], SFR79_grid[i, -1], c=c_mapped, marker='o', label=label)

# # plot residual of directions[0] and directions[1]
# ax[1].plot(x,
#            [(2*(f - b)/(f + b)) if f+b != 0 else 0 for f, b in zip(y_data[0], y_data[1])])
# ax[1].axhline(0, ls=':')

# additional fig and axes config
# fig.suptitle(f'{y_type}: forward vs backward', fontsize=16)
fig.suptitle(f'{y_type}: comparing {len(which_objects)} simulated galaxies', fontsize=16)

# for a in ax:
#     a.invert_xaxis()
#     a.set_xlabel(x_type)
ax[0].invert_xaxis()
ax[0].set_xlabel(x_type)
ax[0].set_yscale('log')
ax[0].set_ylabel(y_type)

ax[1].set_xscale('log')
# ax[1].set_xlabel('simulation number (arb. index)')
ax[1].set_xlabel(c_type)
# ax[1].set_ylabel(f'Δ{y_type}(t) / mean({y_type}' + r'$_\mathrm{fw}$, ' + f'{y_type}' + r'$_\mathrm{bw}$)')
ax[1].set_ylabel(f'log SFR79')
ax[1].text(0.95, 0.05,
           f"{c_type} at lookbacktime = {env_grid[0, c_snp].data['lookbacktime']:.3f}",
           transform=ax[1].transAxes,
           va='bottom', ha='right')

# ax[1].legend()

In [ ]:
fig.savefig(plot_dir / f'comparing_{y_type}_of_{len(which_objects)}_simulated_galaxies'
                       f'_{datetime.now().strftime("%Y.%m.%d-%H.%M.%S")}.png')

**SFR79 from observations vs from simulations**

In [ ]:
# sort simulation results into an array of same shape as observational results
mstar_lower = 10**mstar_mesh[:-1, :-1]
mstar_upper = 10**mstar_mesh[1:, 1:]
sfr_lower = 10**(sfr_mesh[:-1, :-1]) * 10**9  # multiply by 10**9 because of 'pre yr' to 'per Gyr' conversion
sfr_upper = 10**(sfr_mesh[1:, 1:]) * 10**9  # multiply by 10**9 because of 'pre yr' to 'per Gyr' conversion

sfr79_sims_grid = np.full_like(sfr79_medians, np.nan)
# for (sfr79, gal) in zip(SFR79_grid[:, -1], gal_grid[:, -1]):  # special setting if you want the shape at the end
for (sfr79, gal) in zip(SFR79_grid[:, -1], gal_grid[:, 0]):  # regular setting when you want the shape from the start
    w_mstar_lower = mstar_lower < gal.data['mstar']
    w_mstar_upper = mstar_upper > gal.data['mstar']
    w_sfr_lower = sfr_lower < gal.data['SFR']
    w_sfr_upper = sfr_upper > gal.data['SFR']

    w_mstar = w_mstar_lower & w_mstar_upper
    w_sfr = w_sfr_lower & w_sfr_upper

    w = w_mstar & w_sfr

    # w = (mstar_lower < gal.data['mstar'] &
    #      mstar_upper > gal.data['mstar'] &
    #      sfr_lower < gal.data['SFR'] &
    #      sfr_upper > gal.data['SFR'])
    sfr79_sims_grid[w] = sfr79
    # sfr79_sims_grid[w_mstar, w_sfr] = sfr79
    # pass


In [ ]:
# colormap/norm etc. for scatter plot itself
sfr79_range = (-2, 2)
cmap = mpl.cm.RdBu
norm = mpl.colors.Normalize(vmin=sfr79_range[0], vmax=sfr79_range[1])

# set up figure and axes
fig, (ax_obs, ax_sims, ax_cbar) = plt.subplots(1, 3,
                                               gridspec_kw={
                                                   'width_ratios': (9, 9, 1),
                                                   'hspace': 0.05
                                               },
                                               figsize=(21, 9))

# plot observational data
im_obs = ax_obs.pcolormesh(mstar_mesh, sfr_mesh,
                           sfr79_medians,
                           cmap=cmap, norm=norm)

# plot simulation data
im_sims = ax_sims.pcolormesh(mstar_mesh, sfr_mesh,
                            sfr79_sims_grid,
                            cmap=cmap, norm=norm)

# plot colorbar
fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             ax=ax_cbar,
             fraction=0.8,
             extend='both',
             anchor=(0.0, 0.0),
             label='log SFR79')

# adding the Galaxy Main Sequence on top (Saintonge+2016, Eq. 5)
GMS_x = np.linspace(start=np.min(mstar_mesh),
                    stop=np.max(mstar_mesh),
                    num=1000,
                    endpoint=True)
for ax in [ax_obs, ax_sims]:
    ax.plot(GMS_x, GMS_Saintonge2016(GMS_x),
            color='xkcd:magenta', ls=':')

# remove unnecessary axes
ax_cbar.remove()

# figure labelling etc
for ax in [ax_obs, ax_sims]:
    ax.set_xlabel(r'log $M_\star$ [$M_\odot$]')
    ax.set_ylabel(r'log SFR [$M_\odot \, yr^{-1}$]')
ax_sims.text(0.95, 0.05,
             f"min(log SFR79) = {np.nanmin(sfr79_sims_grid):.3f}\n"
             f"max(log SFR79) = {np.nanmax(sfr79_sims_grid):.3f}",
             transform=ax_sims.transAxes,
             va='bottom', ha='right')

In [ ]:
print("min log SFR79 from sims", np.nanmin(sfr79_sims_grid))
print("max log SFR79 from sims", np.nanmax(sfr79_sims_grid))

In [ ]:
# save heat-map plot

fig.savefig(plot_dir / f'SFR79_obs_vs_sims_{datetime.now().strftime("%Y.%m.%d-%H.%M.%S")}.png')